In [1]:
#importing libraies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras
from keras import metrics
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
%matplotlib inline
# Reset TF and random seed (for reproducible results).
tf.keras.backend.clear_session()
tf.random.set_seed(0)

# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "us_perm_visas.csv"]).decode("utf8"))

/home/joshua/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/joshua/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/tmp/ipykernel_2035450/1013195902.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/usr/lib/python3/dist-packages/scipy/__i

us_perm_visas.csv



In [3]:
# Exploratory of Datasets

df = pd.read_csv('us_perm_visas.csv', low_memory = False, parse_dates=['decision_date', 'case_received_date'])

In [4]:
# Displaying number of rows and columns
print('Number of Visa Applications:', len(df))
print('Number of Columns:', len(df.columns))

Number of Visa Applications: 374362
Number of Columns: 154


In [5]:
# Displaying 5 first rows and last 5 rows
print("The first 5 rows:", df.head())
print("The last 5 rows:", df.tail())

The first 5 rows:   add_these_pw_job_title_9089 agent_city agent_firm_name agent_state  \
0                         NaN        NaN             NaN         NaN   
1                         NaN        NaN             NaN         NaN   
2                         NaN        NaN             NaN         NaN   
3                         NaN        NaN             NaN         NaN   
4                         NaN        NaN             NaN         NaN   

  application_type        case_no case_number case_received_date case_status  \
0             PERM  A-07323-97014         NaN                NaT   Certified   
1             PERM  A-07332-99439         NaN                NaT      Denied   
2             PERM  A-07333-99643         NaN                NaT   Certified   
3             PERM  A-07339-01930         NaN                NaT   Certified   
4             PERM  A-07345-03565         NaN                NaT   Certified   

  class_of_admission  ... ri_pvt_employment_firm_to ri_us_workers_co

In [6]:
#display attributes in the dataset.
print(df.columns.values)

['add_these_pw_job_title_9089' 'agent_city' 'agent_firm_name'
 'agent_state' 'application_type' 'case_no' 'case_number'
 'case_received_date' 'case_status' 'class_of_admission'
 'country_of_citizenship' 'country_of_citzenship' 'decision_date'
 'employer_address_1' 'employer_address_2' 'employer_city'
 'employer_country' 'employer_decl_info_title' 'employer_name'
 'employer_num_employees' 'employer_phone' 'employer_phone_ext'
 'employer_postal_code' 'employer_state' 'employer_yr_estab'
 'foreign_worker_info_alt_edu_experience'
 'foreign_worker_info_birth_country' 'foreign_worker_info_city'
 'foreign_worker_info_education' 'foreign_worker_info_education_other'
 'foreign_worker_info_inst' 'foreign_worker_info_major'
 'foreign_worker_info_postal_code' 'foreign_worker_info_rel_occup_exp'
 'foreign_worker_info_req_experience' 'foreign_worker_info_state'
 'foreign_worker_info_training_comp' 'foreign_worker_ownership_interest'
 'foreign_worker_yr_rel_edu_completed' 'fw_info_alt_edu_experience'

In [7]:
# Checking length of case_number and case_no

print("Length of 'case_number' column is: ", len(df['case_number'])," with",df.case_number.isnull().sum(), "missing values")
print("Length of 'case_no' column is: ", len(df['case_no'])," with", df.case_no.isnull().sum(),"missing values \n")

print("First 2 values of case_number column are : \n", df['case_number'].head(2),"\n")

print("Last 2 values of case_number column are : \n", df['case_number'].tail(2), "\n")
print("First 2 values of case_no column are : \n", df['case_no'].head(2), "\n")
print("Last 2 values of case_no column are : \n", df['case_no'].tail(2))

Length of 'case_number' column is:  374362  with 135269 missing values
Length of 'case_no' column is:  374362  with 239093 missing values 

First 2 values of case_number column are : 
 0    NaN
1    NaN
Name: case_number, dtype: object 

Last 2 values of case_number column are : 
 374360    A-16357-84250
374361    A-16279-59292
Name: case_number, dtype: object 

First 2 values of case_no column are : 
 0    A-07323-97014
1    A-07332-99439
Name: case_no, dtype: object 

Last 2 values of case_no column are : 
 374360    NaN
374361    NaN
Name: case_no, dtype: object


In [8]:
# Creating new columns containing non missing values from both "case_number" and "case_no"

casenoindex = df.columns.get_loc("case_no")
casenumberindex = df.columns.get_loc("case_number")
casenumberlist = []

for value in df.iloc[0:135269,casenoindex]:
    casenumberlist.append(value)
    
for value in df.iloc[135269:374363,casenumberindex]:
    casenumberlist.append(value)
    
df['casenumber'] = casenumberlist
df.drop(df.columns[[casenoindex,casenumberindex]], axis=1, inplace=True)

In [9]:
#Printing number of unique values for 'case_status' column

for value in df.case_status.unique():
    print(len(df[df['case_status'] == value])," occurrences of status '{}'".format(value))

181933  occurrences of status 'Certified'
25649  occurrences of status 'Denied'
148586  occurrences of status 'Certified-Expired'
18194  occurrences of status 'Withdrawn'


In [10]:
#Removing all withdrawn applications
df = df[df.case_status != 'Withdrawn']

#Combining certified-expired and certified applications and displaying distribution of "case_status" variable
df.loc[df.case_status == 'Certified-Expired', 'case_status'] = 'Certified'
df.case_status.value_counts()

case_status
Certified    330519
Denied        25649
Name: count, dtype: int64

In [11]:
# Percentage of case_status.

df['case_status'].value_counts()/len(df)*100

case_status
Certified    92.798623
Denied        7.201377
Name: count, dtype: float64

In [12]:
#Dropping all empty columns
df = df.dropna(axis=1, how='all');

#Dropping all empty rows
df = df.dropna(axis=0, how='all');

df.shape

(356168, 153)

In [13]:
# Displaying number of missing values in each column
for column in df.columns:
    print("Attribute '{}' contains ".format(column),  df[column].isnull().sum().sum(), " missing values")

Attribute 'add_these_pw_job_title_9089' contains  317031  missing values
Attribute 'agent_city' contains  153452  missing values
Attribute 'agent_firm_name' contains  157646  missing values
Attribute 'agent_state' contains  156544  missing values
Attribute 'application_type' contains  229320  missing values
Attribute 'case_received_date' contains  126848  missing values
Attribute 'case_status' contains  0  missing values
Attribute 'class_of_admission' contains  21085  missing values
Attribute 'country_of_citizenship' contains  19272  missing values
Attribute 'country_of_citzenship' contains  336951  missing values
Attribute 'decision_date' contains  0  missing values
Attribute 'employer_address_1' contains  37  missing values
Attribute 'employer_address_2' contains  236133  missing values
Attribute 'employer_city' contains  10  missing values
Attribute 'employer_country' contains  126920  missing values
Attribute 'employer_decl_info_title' contains  126886  missing values
Attribute 'em